# Analyzing Education Statistics

### Introduction

To get a sense of [NY State math exam](https://data.cityofnewyork.us/Education/2013-2015-New-York-State-Mathematics-Exam-by-Schoo/gcvr-n8qw).

### Exploring the Dataset

Now that we've identified a candidate dataset where we have close to the population of NY school students represented, let's begin to explore this data.

As a broad first step, we should still aim to check our assumption that this dataset does a good job of capturing the data of NYC city schools.  In checking this we have two questions:

1. Do we have all students from a school?
2. Do we have all schools?

> Of the two questions, which do you think is more important to satisify?   In my opinion, one is more important than the other.

In [1]:
import pandas as pd

df = pd.read_csv('./state_math_exam_by_school.csv')

In [2]:
df[:5]

,DBN,School Name,Grade,Year,Category,Number Tested,Mean Scale Score,# Level 1,% Level 1,# Level 2,% Level 2,# Level 3,% Level 3,# Level 4,% Level 4,# Level 3+4,% Level 3+4
0,01M015,P.S. 015 ROBERTO CLEMENTE,3,2013,All Students,27,278.0,16.0,59.3,11.0,40.7,0.0,0.0,0.0,0.0,0.0,0.0
1,01M015,P.S. 015 ROBERTO CLEMENTE,3,2014,All Students,18,286.0,6.0,33.3,9.0,50.0,2.0,11.1,1.0,5.6,3.0,16.7
2,01M015,P.S. 015 ROBERTO CLEMENTE,3,2015,All Students,17,280.0,10.0,58.8,4.0,23.5,2.0,11.8,1.0,5.9,3.0,17.6
3,01M015,P.S. 015 ROBERTO CLEMENTE,4,2013,All Students,20,277.0,12.0,60.0,6.0,30.0,1.0,5.0,1.0,5.0,2.0,10.0
4,01M015,P.S. 015 ROBERTO CLEMENTE,4,2014,All Students,24,282.0,9.0,37.5,14.0,58.3,1.0,4.2,0.0,0.0,1.0,4.2


### Clean and Reduce our Data

Before answering these questions, let's begin to reduce, and then clean some of our data.

To start, we may be able to get rid of some columns.  The # Level, and % Level is repeated information.  If we have the number and the total, we can calculate the level.

Start by collecting all of the percent columns, then we'll drop them.

In [3]:
percent_cols = [col for col in df.columns if '%' in col]
percent_cols
# ['% Level 1', '% Level 2', '% Level 3', '% Level 4', '% Level 3+4']

['% Level 1', '% Level 2', '% Level 3', '% Level 4', '% Level 3+4']

In [4]:
df_no_percent = df.drop(columns = percent_cols)

In [5]:
df_no_percent[:3]

,DBN,School Name,Grade,Year,Category,Number Tested,Mean Scale Score,# Level 1,# Level 2,# Level 3,# Level 4,# Level 3+4
0,01M015,P.S. 015 ROBERTO CLEMENTE,3,2013,All Students,27,278.0,16.0,11.0,0.0,0.0,0.0
1,01M015,P.S. 015 ROBERTO CLEMENTE,3,2014,All Students,18,286.0,6.0,9.0,2.0,1.0,3.0
2,01M015,P.S. 015 ROBERTO CLEMENTE,3,2015,All Students,17,280.0,10.0,4.0,2.0,1.0,3.0


It looks like the `Category` column may just consist of one value, `All Students`.  Use value_counts to see if this is true.

In [6]:
df_no_percent['Category'].value_counts()

All Students    14315
Not SWD         14296
EP              14289
Female          14259
Hispanic        14246
Male            14227
SWD             14099
Black           13757
ELL             13251
Former ELL      12156
White           11393
Asian           11115
Name: Category, dtype: int64

Apparently, this data is also broken down according to demographic information.  So let's keep this column.  The `# Level 3+4` can go though - it's just a sum of the previous two columns.  Let's drop it.

In [7]:
reduced_df = df_no_percent.iloc[:, :-1]

In [8]:
reduced_df[:3]

,DBN,School Name,Grade,Year,Category,Number Tested,Mean Scale Score,# Level 1,# Level 2,# Level 3,# Level 4
0,01M015,P.S. 015 ROBERTO CLEMENTE,3,2013,All Students,27,278.0,16.0,11.0,0.0,0.0
1,01M015,P.S. 015 ROBERTO CLEMENTE,3,2014,All Students,18,286.0,6.0,9.0,2.0,1.0
2,01M015,P.S. 015 ROBERTO CLEMENTE,3,2015,All Students,17,280.0,10.0,4.0,2.0,1.0


Ok, now let's see if our data has the correct types.

In [9]:
reduced_df.dtypes 

# DBN                  object
# School Name          object
# Grade                object
# Year                  int64
# Category             object
# Number Tested         int64
# Mean Scale Score    float64
# # Level 1           float64
# # Level 2           float64
# # Level 3           float64
# # Level 4           float64
# dtype: object

DBN                  object
School Name          object
Grade                object
Year                  int64
Category             object
Number Tested         int64
Mean Scale Score    float64
# Level 1           float64
# Level 2           float64
# Level 3           float64
# Level 4           float64
dtype: object

The only column that potentially looks like it can be improved is the `Grade` argument.  Let's take a deeper look at it.

In [10]:
df['Grade'].value_counts()

# All Grades    38978
# 3             25139
# 4             24541
# 5             24326
# 6             16905
# 7             16165
# 8             15349
# Name: Grade, dtype: int64

All Grades    38978
3             25139
4             24541
5             24326
6             16905
7             16165
8             15349
Name: Grade, dtype: int64

We can see that in addition grades 3 through 8 it also has an `All Grades` category.  So our dtype of object looks fine.

### Is it representative?

Now that we have spent some time reducing and cleaning our data, let's see if our data is comprehensive.  Remember, that we broke this down into two questions:

1. Do we have all students from a school?
2. Do we have all schools?

Let's start by looking to see if all students (or roughly all students) are included.

To do this, we should get population student amounts for different schools, or at least a selection of our schools.  Let's focus on middle schools.  To get our middle schools, we can get a list of schools that have a grade 8.  Because we have entries for every row, we can further filter this by records from 2015, the most recent year.

In [11]:
middle_schools = reduced_df[(reduced_df['Grade'] == '8') & (reduced_df['Year'] == 2015)]


In [12]:
middle_schools[:3]

,DBN,School Name,Grade,Year,Category,Number Tested,Mean Scale Score,# Level 1,# Level 2,# Level 3,# Level 4
53,01M034,P.S. 034 FRANKLIN D. ROOSEVELT,8,2015,All Students,59,298.0,21.0,24.0,9.0,5.0
136,01M140,P.S. 140 NATHAN STRAUS,8,2015,All Students,58,285.0,28.0,26.0,4.0,0.0
169,01M184,P.S. 184M SHUANG WEN,8,2015,All Students,56,360.0,0.0,4.0,13.0,39.0


Let's take a look at how many schools we have now.

In [13]:
middle_schools.shape

(5170, 11)

This seems like a lot.  Is it?  Let's use Google to get a rough number of the number of middle schools in NYC.

According [to the DOE](https://www.schools.nyc.gov/about-us/reports/doe-data-at-a-glance):

> There are 1,866 schools within the DOE as of September 2019, including 260 charter schools.  

So we must have some duplicated data.  Find out the source of it, and remove it.

In [14]:
ms_all_df = middle_schools[middle_schools['Category'] == 'All Students']

In [15]:
ms_all_df.shape

# (472, 11)

(472, 11)

Ok, this seems more reasonable.

So currently we have 472 schools in our dataframe.  Now is that all inclusive?  We can cross check this, with the [2015-2016 class size report for NYC high schools and middle schools](https://data.cityofnewyork.us/Education/2015-2016-Final-Class-Size-Report-Middle-High-Scho/icps-nwdu).  Let's load it up.

In [70]:
import pandas as pd
url = '2014-2015_Class_Size_Report_School_Level_Detail_Summary.csv'

df = pd.read_csv('./15-16-class-size-middle.csv')


In [71]:
df[:3]

,DBN,School Name,Grade Level,Program Type,Department,Subject,Number of Students,Number of Classes,Average Class Size
0,01M034,P.S. 034 FRANKLIN D. ROOSEVELT,MS Core,Gen Ed,English,MS English Core,85,3,28.3
1,01M034,P.S. 034 FRANKLIN D. ROOSEVELT,MS Core,Gen Ed,Math,MS Math Core,85,3,28.3
2,01M034,P.S. 034 FRANKLIN D. ROOSEVELT,MS Core,Gen Ed,Science,MS Science Core,85,3,28.3


Find the unique number of middle schools.

> You did it correctly if you get a number at or near 502.

In [18]:
unique_dbns = df[df['Grade Level'] == 'MS Core']['DBN'].unique()

In [19]:
unique_dbns.shape

(502,)

Remember our original data had 472.  So this isn't bad.  
> It could be due to difference between 2014-2015 data and 2015-2016 data.

2. Checking the number of students

Now let's try to get a sense of if our exam data includes almost all of the students.  Let's start with a dataframe of all rows pertaining to a middle school.  
> This should have each middle school listed multiple times at this point. 

In [20]:
ms_df_class_size = df[df['Grade Level'] == 'MS Core']

In [21]:
ms_df_class_size.shape

(5731, 9)

In [22]:
ms_df_class_size[:3]

,DBN,School Name,Grade Level,Program Type,Department,Subject,Number of Students,Number of Classes,Average Class Size
0,01M034,P.S. 034 FRANKLIN D. ROOSEVELT,MS Core,Gen Ed,English,MS English Core,85,3,28.3
1,01M034,P.S. 034 FRANKLIN D. ROOSEVELT,MS Core,Gen Ed,Math,MS Math Core,85,3,28.3
2,01M034,P.S. 034 FRANKLIN D. ROOSEVELT,MS Core,Gen Ed,Science,MS Science Core,85,3,28.3


Now it looks like the number of students information is duplicated, if it is, then this likely pertains to the number of students in the entire school.  

Let's see if every school listing has the number of students repeated.

[Stackoverflow post](https://stackoverflow.com/questions/53950883/check-if-all-elements-in-a-group-are-equal-using-pandas-groupby)

In [23]:
ms_df_class_size.groupby(['DBN'])['Number of Students'].nunique()[:3]

DBN
01M034    3
01M140    7
01M184    5
Name: Number of Students, dtype: int64

Huh it looks like they do not.  Let's look at just school `'01M034'`.

In [121]:
# ms_df_class_size[ms_df_class_size['DBN'] == '01M034']

It seems like there are different numbers depending on the program type.  Let's just select the max number of students per school for now.

In [50]:
max_class_size = ms_df_class_size.groupby('DBN')['Number of Students'].max()

In [51]:
max_class_size[:3]

# 01M034     86
# 01M140    118
# 01M184    109
# Name: Number of Students, dtype: int64

DBN
01M034     86
01M140    118
01M184    109
Name: Number of Students, dtype: int64

In [27]:
max_class_size.shape

# (502,)

(502,)

Ok, now we can add try to combine this information with our school data dataframe.  

Let's go back to our dataframe with students.

In [28]:
reduced_df[:3]

,DBN,School Name,Grade,Year,Category,Number Tested,Mean Scale Score,# Level 1,# Level 2,# Level 3,# Level 4
0,01M015,P.S. 015 ROBERTO CLEMENTE,3,2013,All Students,27,278.0,16.0,11.0,0.0,0.0
1,01M015,P.S. 015 ROBERTO CLEMENTE,3,2014,All Students,18,286.0,6.0,9.0,2.0,1.0
2,01M015,P.S. 015 ROBERTO CLEMENTE,3,2015,All Students,17,280.0,10.0,4.0,2.0,1.0


Select the schools that have grades 6, 7, or 8.

In [30]:
import numpy as np
middle_schools_selected = reduced_df[np.isin(reduced_df['Grade'], ['6', '7', '8'])]

Then get to the year 2015.

In [34]:
ms_2015 = middle_schools_selected[middle_schools_selected['Year'] == 2015]

In [37]:
ms_2015_all_df = ms_2015[ms_2015['Category'] == 'All Students']

This number makes sense because we have three grades per school.

In [38]:
ms_2015_all_df.shape

(1470, 11)

In [41]:
number_tested = ms_2015_all_df.groupby('DBN').sum()['Number Tested']

In [56]:
ms_2015_all_df[ms_2015_all_df['DBN'] == '01M034']

,DBN,School Name,Grade,Year,Category,Number Tested,Mean Scale Score,# Level 1,# Level 2,# Level 3,# Level 4
47,01M034,P.S. 034 FRANKLIN D. ROOSEVELT,6,2015,All Students,56,295.0,19.0,23.0,11.0,3.0
50,01M034,P.S. 034 FRANKLIN D. ROOSEVELT,7,2015,All Students,43,296.0,17.0,15.0,10.0,1.0
53,01M034,P.S. 034 FRANKLIN D. ROOSEVELT,8,2015,All Students,59,298.0,21.0,24.0,9.0,5.0


In [72]:
num_students = pd.concat([max_class_size, number_tested], axis=1)

/Users/jeff/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  """Entry point for launching an IPython kernel.


In [73]:
num_students[:3]

,Number of Students,Number Tested
01M034,86.0,158.0
01M140,118.0,164.0
01M184,109.0,153.0


It looks like our number of students could be off.  Let's go back to our middle school class size data.  Let's look at it unaggregated, and show enough rows for two schools.

In [100]:
# ms_df_class_size[ms_df_class_size['DBN'] == '01M450']

It looks like instead of just taking the max number per school we should group each school by the program type, and then take the max of that.  (This won't be perfect, but it looks close.)

In [87]:
grouped_df_dbn_program = ms_df_class_size.groupby(['DBN', 'Program Type']).max()

In [90]:
grouped_df_dbn_program[:6]

School Name Grade Level  \
DBN    Program Type                                               
01M034 Gen Ed        P.S. 034 FRANKLIN D. ROOSEVELT     MS Core   
       ICT           P.S. 034 FRANKLIN D. ROOSEVELT     MS Core   
       SC            P.S. 034 FRANKLIN D. ROOSEVELT     MS Core   
01M140 Gen Ed                P.S. 140 NATHAN STRAUS     MS Core   
       ICT                   P.S. 140 NATHAN STRAUS     MS Core   
       SC                    P.S. 140 NATHAN STRAUS     MS Core   

                         Department                 Subject  \
DBN    Program Type                                           
01M034 Gen Ed        Social Studies  MS Social Studies Core   
       ICT           Social Studies  MS Social Studies Core   
       SC            Social Studies  MS Social Studies Core   
01M140 Gen Ed        Social Studies  MS Social Studies Core   
       ICT           Social Studies  MS Social Studies Core   
       SC            Social Studies  MS Social Studies Core   

                     Number of Students  Number of Classes  Average Class Size  
DBN    Program Type                                                             
01M034 Gen Ed                        85                  3                28.3  
       ICT                           86                  3                28.7  
       SC                             6                  1                 6.0  
01M140 Gen Ed                       118                  4                29.5  
       ICT                           63                  2                31.5  
       SC                            19                  3                 6.3

From this, we really just need the number of students.

In [95]:
school_size_est = grouped_df_dbn_program['Number of Students'].groupby('DBN').sum()

In [101]:
combined_students_tested = pd.concat([number_tested, school_size_est], axis = 1)

/Users/jeff/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  """Entry point for launching an IPython kernel.


In [108]:
combined_students_tested[:3]

,Number Tested,Number of Students
01M034,158.0,177.0
01M140,164.0,200.0
01M184,153.0,269.0


In [110]:
diff = combined_students_tested['Number of Students'] - combined_students_tested['Number Tested']

In [111]:
combined_students_tested['difference'] = diff

In [114]:
pct_diff = combined_students_tested['difference']/combined_students_tested['Number of Students']

In [115]:
combined_students_tested['pct_diff'] = pct_diff

In [118]:
combined_students_tested['pct_diff'].describe()

count    499.000000
mean       0.192855
std        0.402554
min       -7.213115
25%        0.088975
50%        0.204640
75%        0.334891
max        0.817420
Name: pct_diff, dtype: float64